In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [2]:
!git clone https://github.com/rodrigorivera/mds20_deepfolio

from mds20_deepfolio.models.NeuralHawkesProcess.DataWrapper import LOBDataset, collate_fn
from mds20_deepfolio.models.NeuralHawkesProcess.model import NHPModel
from mds20_deepfolio.models.NeuralHawkesProcess.train import train

!unzip /content/mds20_deepfolio/models/NeuralHawkesProcess/data/LOB.zip \
      -d /content/mds20_deepfolio/models/NeuralHawkesProcess/data/


Cloning into 'mds20_deepfolio'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 736 (delta 3), reused 0 (delta 0), pack-reused 726
Receiving objects: 100% (736/736), 69.98 MiB | 22.22 MiB/s, done.
Resolving deltas: 100% (425/425), done.
Archive:  /content/mds20_deepfolio/models/NeuralHawkesProcess/data/LOB.zip
  inflating: /content/mds20_deepfolio/models/NeuralHawkesProcess/data/XRP.npy  
  inflating: /content/mds20_deepfolio/models/NeuralHawkesProcess/data/ETH.npy  


# **ETH Train**

In [95]:
ETH_dataset = LOBDataset('/content/mds20_deepfolio/models/NeuralHawkesProcess/data/ETH.npy')
data_len = len(ETH_dataset)
print(data_len, ETH_dataset[0])

527 (tensor([14.2720, 13.3890, 27.3870,  ..., 11.3620, 23.3570,  7.3450]), tensor([1, 0, 0,  ..., 0, 0, 1]))


In [51]:
train_dataset = torch.utils.data.Subset(ETH_dataset, range(395))


(tensor([14.2720, 13.3890, 27.3870,  ..., 11.3620, 23.3570,  7.3450]),
 tensor([1, 0, 0,  ..., 0, 0, 1]))

In [57]:
range(395)

range(0, 395)

In [96]:
train_len = int(data_len*0.75)
val_len = int(data_len*0.125)
test_len = data_len - train_len - val_len

#train_dataset, val_dataset,test_dataset = torch.utils.data.random_split(ETH_dataset, [train_len, val_len, test_len])
train_dataset = torch.utils.data.Subset(ETH_dataset, range(395))
val_dataset = torch.utils.data.Subset(ETH_dataset, range(395,395+66))
test_dataset = torch.utils.data.Subset(ETH_dataset, range(395+66,395+66+66))

train_loader = DataLoader(train_dataset, batch_size=12, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=12, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=12, collate_fn=collate_fn)

print('lenght of train_dataset:', len(train_dataset))
print('lenght of val_dataset:', len(val_dataset))
print('lenght of test_dataset:', len(test_dataset))

lenght of train_dataset: 395
lenght of val_dataset: 66
lenght of test_dataset: 66


In [98]:
DO_TRAIN = False
model = NHPModel(256, device=device).to(device)

if DO_TRAIN:
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    statiscs = train(model, optimizer, train_loader, val_loader, device, n_epochs = 25, sum_losses=True)
else:
    model.load_state_dict(torch.load('/content/mds20_deepfolio/models/NeuralHawkesProcess/weights/ETH,256.pth'))

# **Evaluate model using Linear layer for time and type prediction**

In [102]:
from sklearn.metrics import accuracy_score

loss_time, type_acc,loss_llh = 0, 0,0
for event_seq, time_seq in test_loader:
    event_seq, time_seq = event_seq.to(device), time_seq.to(device)
    intens, time, event = model.forward(event_seq, time_seq)

    loss_llh += model.LogLikelihoodLoss(intens, time_seq) / (time_seq.shape[0] * time_seq.shape[1])
    loss_time += model.time_loss(time, time_seq)
    type_acc += accuracy_score(event[:,:-1].argmax(dim=2).cpu().reshape(-1), 
                                                  event_seq[:, 1:].cpu().reshape(-1))



In [106]:
print('Time RMSE on test dataset', (loss_time/len(test_loader)).item()**0.5)
print('Type prediction accuracy on test dataset', type_acc/len(test_loader))
print('Log-likelihood on test dataset', -(loss_llh/len(test_loader)).item())

Time RMSE on test dataset 17.859185325436354
Type prediction accuracy on test dataset 0.5726342592592591
Log-likelihood on test dataset -8.170878410339355


# **Evaluate model using probability function for time and type prediction**

In [107]:
from sklearn.metrics import accuracy_score, mean_squared_error

def evaluate_prediction(model, dataloader, device):
        """
        Evalute prediction on give dataset
        Will compute mse and accuracy score for event time and type prediction.
        Input:
           model - NHP model to compute decay states for sequence
           dataloader - dataloader with data
        Output:
           mean_squared_error - for event time prediction
           accuracy_score - for event type prediction
        """
        pred_data = []
        for event_seqs, time_seqs in dataloader:
            for i in range(len(event_seqs)):
                pred_data.append(predict_event(model, time_seqs[i], event_seqs[i], len(time_seqs[i])-1, device)[:4])

        pred_data = np.array(pred_data)
        time_gt, time_pred = pred_data[:,0], pred_data[:,1]
        type_gt, type_pred = pred_data[:,2], pred_data[:,3]

        time_mse_error = mean_squared_error(time_gt, time_pred)
        type_accuracy = accuracy_score(type_gt, type_pred)

        return time_mse_error, type_accuracy

def predict_event(model, seq_time, seq_events, seq_length, device, hmax = 40,
                     n_samples=1000):
        """ 
        Predict last event time and type for the given sequence 
        Last event takes as unknown and model feeds with all remain sequence.
        Input:
            model - NHP model to compute decay states for sequence
            seq_time - torch.tensor with time diffs between events in sequence
            seq_events - torch.tensor with event types for each time point
            seq_length - length of the sequence
        
        Output:
            pred_dt - predicted dt for next event
            gt_dt - gt df of next event
            pred_type - predicted type of next event
            gt_type - gt_type of next event
            time_between_events - np.array - generated timestamps
            intensity - np.array - intensity after event
        """

        """ Feed the model with sequence and compute decay cell state """

        with torch.no_grad():
            model.init_states(1)
            for i in range(seq_length):
                c_t, c_target, output, decay = model.CTLSTM_cell(model.Embedding(seq_events[i].to(device)).unsqueeze(0), model.hidden_decay, 
                                                               model.cell_decay, model.cell_target)

                if i < seq_length - 1:

                    c_t = c_t * torch.exp(-decay * seq_time[i, None].to(device)) 
                    h_t = output * torch.tanh(c_t)

            # gt last and one before last event types and times
            last_type, gt_type = seq_events[i], seq_events[i + 1]
            gt_dt = seq_time[i]


            """ Make prediction for the next event time and type """
            model.eval()
            timestep = hmax / n_samples

            # 1) Compute intensity
            time_between_events = torch.linspace(0, hmax, n_samples + 1).to(device)
            hidden_vals = h_t * torch.exp(-decay * time_between_events[:, None])
            intensity = model.intensity_layer(hidden_vals.to(device))
            intensity_sum = intensity.sum(dim=1)


            # 2) Compute density via integral 
            density = torch.cumsum(timestep * intensity.sum(dim=1), dim=0)
            density = intensity_sum * torch.exp(-density)

            # 3) Predict time of the next event via trapeze method
            t = time_between_events * density   
            pred_dt = (timestep * 0.5 * (t[1:] + t[:-1])).sum() 
            # 4) Predict type of the event via trapeze method
            P = intensity / intensity_sum[:, None] * density[:, None]  
            pred_type = torch.argmax(timestep * 0.5 * (P[1:] + P[:-1])).sum(dim=0)

            return pred_dt.cpu().numpy(), gt_dt.cpu().numpy(), pred_type.cpu().numpy(), gt_type.cpu().numpy(), \
                            time_between_events.cpu().numpy(), intensity.cpu().numpy()


In [90]:
time_mse_error_test, type_accuracy_test = evaluate_prediction(model, test_loader,device)

In [108]:
print('Time RMSE on test dataset', time_mse_error_test**0.5)
print('Type prediction accuracy on test dataset', type_accuracy_test)

Time RMSE on test dataset 36.55562393610459
Type prediction accuracy on test dataset 0.42424242424242425
